# Euclidean distance between two images

In [ ]:
import ee
ee.Initialize()

In [ ]:
from geetools import tools, algorithms

In [ ]:
bands = ['B1','B2','B3']

In [ ]:
image1 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_232089_20170126').select(bands)

In [ ]:
image2 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_232089_20170211').select(bands)

In [ ]:
image3 = ee.Image('LANDSAT/LC08/C02/T1_L2/LC08_232089_20170227').select(bands)

In [ ]:
p = ee.Geometry.Point(-72, -42)

In [ ]:
data1 = tools.image.getValue(image1, p, 30, side='client')

In [ ]:
data1

In [ ]:
data2 = tools.image.getValue(image2, p, 30, side='client')

In [ ]:
data2

In [ ]:
data3 = tools.image.getValue(image3, p, 30, side='client')

In [ ]:
data3

In [ ]:
distance = algorithms.euclideanDistance(image1, image2)

In [ ]:
data_dist = tools.image.getValue(distance, p, 30, side='client')

In [ ]:
data_dist

## Compute locally to compare

In [ ]:
import math

In [ ]:
# Euclidean distance
def distanceF(arr1, arr2):
    zipped = zip(arr1, arr2)
    accum = 0
    for a, b in zipped:
        rest = (a-b)
        calc = rest*rest
        accum += calc
    return math.sqrt(accum)

In [ ]:
arr1 = data1.values()
arr1

In [ ]:
arr2 = data2.values()
arr2

In [ ]:
local_distance = distanceF(arr1, arr2)

In [ ]:
local_distance

# Distance sum between one image and a collection of images

In [ ]:
col = ee.ImageCollection.fromImages([image2, image3])

In [ ]:
summed = algorithms.sumDistance(image1, col)

In [ ]:
summed_data = tools.image.getValue(summed, p, 30, side='client')

In [ ]:
summed_data

## Compute locally to compare results

In [ ]:
from copy import copy
# Sum of distances
def sum_distances(values):
    results = {}
    for i, val in enumerate(values):
        val = list(val)
        cop = copy(values)
        cop = [list(a) for a in cop]
        cop.remove(val)
        dist = 0
        for r in cop:
            r = list(r)
            d = distanceF(val, r)
            dist += d
        results[i] = dist

    return results

In [ ]:
values = [list(data1.values()), list(data2.values()), list(data3.values())]

In [ ]:
values

In [ ]:
sum_distances(values)

## Handling mask values

In [ ]:
masked_image = image2.updateMask(image2.gte(0).Not())

In [ ]:
data_masked = tools.image.getValue(masked_image, p, 30, side='client')
data_masked

In [ ]:
data1

In [ ]:
distance_masked = algorithms.euclideanDistance(image1, masked_image)

In [ ]:
data_dist_masked = tools.image.getValue(distance_masked, p, 30, side='client')
data_dist_masked

In [ ]:
distanceF(data1.values(), [0, 0, 0])

In [ ]:
outside = image1
inside = masked_image

In [ ]:
col_masked = ee.ImageCollection.fromImages([inside])

In [ ]:
summed_masked = algorithms.sumDistance(outside, col_masked)

In [ ]:
summed_data_masked = tools.image.getValue(summed_masked, p, 30, side='client')

In [ ]:
summed_data_masked